## The following are the codes demonstrating the performance of CCSF for dimensionality reduction, visualization, clustering, and temporal and spatial mapping of gene expression data

In [4]:
# Import all the necessary Python packages
from ccsf import CCSF

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
from scipy import stats
import numpy as np
import novosparc
import random

%matplotlib inline

rngState=0 # For reproducibility

In [ ]:

######################################################################
## 1. Examples of learning the best metric and number of data classes##
######################################################################
# Load TCGA data
data = pd.read_csv('../data/tcga_data.csv', header=None,
                   delim_whitespace=False)
label = pd.read_csv('../data/tcga_label.csv', header=None,
                         delim_whitespace=False).to_numpy()

data=data.to_numpy()
n=2000 # Select 2000 points randomly from the data for learning the distance metric.
manifolder = CCSF(random_state=rngState)
random.seed(a=rngState) 
sampleix = random.sample( range( data.shape[0] ), int(n) )
dataSampled=data[sampleix]
numInitCls=int(np.floor(data.shape[0]/500))
dataSampled = stats.zscore(dataSampled, axis=0, ddof=1)
metric,numClass = manifolder.metric_learning(dataSampled, verbose=False)
print('metric for cMAP is:', metric)
# # You are supposed to see 'correlation' as the result.

##############################################
## 2. Example of cMAP #########
##############################################
# Prepare the data for input to cMAP
feed_data = {'data': data}
num_comp=np.array([numClass-1]) # The number of CDM components to be used
# to compute the cMAP
manifolder = CCSF(n_clusters=numClass, num_comp=num_comp,metric=metric)
embedding_CMAP = manifolder.cMap(data=feed_data)
embedding = embedding_CMAP[0]
plt.figure()
plt.title('cMAP visualization ')
plt.scatter(embedding[:,0],embedding[:,1],c=label.T,s=0.5)
plt.xlabel("cMAP1")
plt.ylabel("cMAP2")
plt.show()  

In [5]:
##############################################
## 3. Example of temporal mapping by CCSF #########
##############################################
data = pd.read_csv('../data/organoidData.csv', header=None,
                    delim_whitespace=False)
label = pd.read_csv('../data/organoidDataLabel.csv', header=None,
                          delim_whitespace=False).to_numpy()
data=data.to_numpy()

# Learning the metric for cPHATE
n=2000
manifolder = CCSF(random_state=rngState)
random.seed(a=rngState) 
sampleix = random.sample( range( data.shape[0] ), int(n) )
dataSampled=data[sampleix]
numInitCls=int(np.floor(data.shape[0]/500))
dataSampled = stats.zscore(dataSampled, axis=0, ddof=1)
metric,numClass = manifolder.metric_learning(dataSampled, verbose=False)
print('metric for cPHATE is:', metric)

numClass=33 # Fixed for cellular trajectory analysis tasks
data = stats.zscore(data, axis=0, ddof=1)
num_comp=np.array([numClass-1]) # Number of CCSF components
manifolder = CCSF(n_clusters=numClass, num_comp=num_comp,metric=metric,random_state=rngState)

# cPHATE
feed_data = {'data': data}
embedding_CPHATE = manifolder.cPHATE(data=feed_data)

embedding = embedding_CPHATE[0]
plt.figure()
plt.title('cPHATE for 32 CCIF components ')
plt.scatter(embedding[:,0],embedding[:,1],c=label.T,s=0.5)
plt.xlabel("cPHATE1")
plt.ylabel("cPHATE2")
plt.show() 


In [ ]:

##############################################
## 4. Example of spatial mapping by CCSF #########
##############################################

# Read the BDTNP database
print ('Loading data ... ', end='', flush=True)
gene_names = np.genfromtxt('novosparc/datasets/bdtnp/dge.txt', usecols=range(84),
                          dtype='str', max_rows=1)
dge = np.loadtxt('novosparc/datasets/bdtnp/dge.txt', usecols=range(84), skiprows=1)
# Optional: downsample number of cells
cells_selected, dge = novosparc.pp.subsample_dge(dge, 3039, 3040)
num_cells = dge.shape[0]    
data=dge
feed_data = {'data': data}

# Learning the metric for cSPARC
n=2000
random.seed(a=rngState)
sampleix = random.sample( range( data.shape[0] ), int(n) )
dataSampled=data[sampleix]
manifolder = CCSF(random_state=rngState)
dataSampled = stats.zscore(dataSampled, axis=0, ddof=1)
numInitCls=int(np.floor(data.shape[0]/500))
metric,numClass = manifolder.metric_learning(dataSampled, verbose=False)
print('metric for CSPARC is:', metric)

print ('Reading the target space ... ', end='', flush=True)    
    # Read and use the bdtnp geometry
locations = np.loadtxt('novosparc/datasets/bdtnp/geometry.txt', usecols=range(3), skiprows=1)
locations = locations[:, [0, 2]]
locations = locations[cells_selected, :] # downsample to the cells selected above

# Compute the spatial maps of the genes
manifolder = CCSF(n_clusters=numClass,num_comp=numClass-1,metric=metric)        
embedding_CSPARC = manifolder.cSpaRc(data=feed_data,locations=locations)


gene='ftz'
#gene='sna' Try other gene in place of ftz

d=embedding_CSPARC[np.argwhere(gene_names == gene), :].flatten()
plt.figure()
plt.title('cSpaRc spatial reconstruction of ftz gene')
plt.scatter(locations[:,0],locations[:,1],c=d.T)
plt.show()